In [11]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [1]:
from sentence_transformers import SentenceTransformer

d:\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

d:\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Abhishek Mukherjee\.cache\huggingface\hub\models--sentence-transformers--multi-qa-MiniLM-L6-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
v=model.encode('I just discovered a course. Can I still join it?')

In [5]:
v.dot(v)

1.0000002

In [7]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector":{
                "type":"dense_vector",
                "dims":384,
                "index":True,
                "similarity":"cosine"
            },
            "text_vector":{
                "type":"dense_vector",
                "dims":384,
                "index":True,
                "similarity":"cosine"
            },
            "question_text_vector":{
                "type":"dense_vector",
                "dims":384,
                "index":True,
                "similarity":"cosine"
            }
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)



ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
from tqdm.auto import tqdm

In [12]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|██████████| 948/948 [04:03<00:00,  3.89it/s]


In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [01:06<00:00, 14.31it/s]


In [14]:
query = 'I just discovered the course. Can I still join it?'

In [15]:
v_q=model.encode(query)

In [16]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [18]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [19]:
import pandas as pd

In [20]:
ground_truth=pd.read_csv('ground-truth-data.csv')

In [21]:
ground_truth=ground_truth.to_dict(orient='records')

In [25]:
def hit_rate(relevance_total):
    cnt=0
    for line in relevance_total:
        if True in line:
            cnt+=1
    return cnt/len(relevance_total)

In [26]:
def mrr(relevance_total):
    total_score=0
    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank]== True:
                total_score=total_score+1/(rank+1)
    return total_score/len(relevance_total)

In [23]:
def evaluate(ground_truth,search_function):
    relevance_total=[]

    for q in tqdm(ground_truth):
        doc_id=q['document']
        results=search_function(q)
        relevance=[d['id']==doc_id for d in results]
        relevance_total.append(relevance)
    
    return {
        'hit_rate':hit_rate(relevance_total),
        'mrr':mrr(relevance_total)
    }


In [27]:
evaluate(ground_truth,question_vector_knn)

100%|██████████| 317/317 [00:29<00:00, 10.67it/s]


{'hit_rate': 0.7413249211356467, 'mrr': 0.6175078864353313}

better than text search when compared both metrics but text search was also much faster

In [28]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [29]:
evaluate(ground_truth,text_vector_knn)

100%|██████████| 317/317 [00:23<00:00, 13.75it/s]


{'hit_rate': 0.804416403785489, 'mrr': 0.6951104100946374}

text vector even better

In [30]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [31]:
evaluate(ground_truth,question_text_vector_knn)

100%|██████████| 317/317 [01:51<00:00,  2.83it/s]


{'hit_rate': 0.8738170347003155, 'mrr': 0.7775499474237647}

question_text_vector even better than text_vector